In [9]:
!pip install ipywidgets==7.0.0 --quiet

In [2]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
model_id, model_version = "huggingface-eqa-distilbert-base-uncased", "*"

In [4]:
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

# model_version="*" fetches the latest version of the model.
infer_model_id, infer_model_version = "huggingface-eqa-distilbert-base-uncased", "*"

endpoint_name = name_from_base(f"insight-example-{infer_model_id}")

inference_instance_type = "ml.p2.xlarge"

# Retrieve the inference docker container uri.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=infer_model_id,
    model_version=infer_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri.
deploy_source_uri = script_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, script_scope="inference"
)
# Retrieve the base model uri.
base_model_uri = model_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, model_scope="inference"
)
# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
)
# deploy the Model.
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
---------!

In [5]:
question_context1 = [
    "What is Southern California often abbreviated as?",
    "Southern California, often abbreviated SoCal, is a geographic and cultural region that generally comprises California's southernmost 10 counties. The region is traditionally described as "
    "eight counties"
    ", based on demographics and economic ties: Imperial, Los Angeles, Orange, Riverside, San Bernardino, San Diego, Santa Barbara, and Ventura. The more extensive 10-county definition, including Kern and San Luis Obispo counties, is also used based on historical political divisions. Southern California is a major economic center for the state of California and the United States.",
]
question_context2 = [
    "Who directed Spectre?",
    "Spectre (2015) is the twenty-fourth James Bond film produced by Eon Productions. It features Daniel Craig in his fourth performance as James Bond, and Christoph Waltz as Ernst Stavro Blofeld, with the film marking the character's re-introduction into the series. It was directed by Sam Mendes as his second James Bond film following Skyfall, and was written by John Logan, Neal Purvis, Robert Wade and Jez Butterworth. It is distributed by Metro-Goldwyn-Mayer and Columbia Pictures. With a budget around $245 million, it is the most expensive Bond film and one of the most expensive films ever made.",
]

In [7]:
question_context1 = [
    "Who is in the meeting ?",
    "Team,here is my update:1) I am fighting a bad cold (hopefully not flu) that has slowed me down a bit the last 2 days 2) I am progressing on the clickable prototype and am really close to getting it to a stopping point.  I will likely need some more time this week to complete that task, assuming I can shake this illness. 3) I can attend Wednesday zoom tech meeting until 930s "
    ,
]
question_context2 = [
    "Who directed Spectre?",
    "Spectre (2015) is the twenty-fourth James Bond film produced by Eon Productions. It features Daniel Craig in his fourth performance as James Bond, and Christoph Waltz as Ernst Stavro Blofeld, with the film marking the character's re-introduction into the series. It was directed by Sam Mendes as his second James Bond film following Skyfall, and was written by John Logan, Neal Purvis, Robert Wade and Jez Butterworth. It is distributed by Metro-Goldwyn-Mayer and Columbia Pictures. With a budget around $245 million, it is the most expensive Bond film and one of the most expensive films ever made.",
]

In [ ]:
from transformers import pipeline

In [8]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"


def query_endpoint(encoded_text):
    response = base_model_predictor.predict(
        encoded_text, {"ContentType": "application/list-text", "Accept": "application/json;verbose"}
    )
    return response


def parse_response(query_response):
    model_predictions = json.loads(query_response)
    answer = (model_predictions["answer"],)
    return answer


for question_context in [question_context1, question_context2]:
    query_response = query_endpoint(json.dumps(question_context).encode("utf-8"))
    answer = parse_response(query_response)
    print(
        f"Inference:{newline}"
        f"Question: {bold}{question_context[0]}{unbold}{newline}"
        f"Context: {question_context[1]}{newline}"
        f"model answer: {bold}{answer}{unbold}{newline}"
    )

Inference:
Question: Who is in the meeting ?
Context: Team,here is my update:1) I am fighting a bad cold (hopefully not flu) that has slowed me down a bit the last 2 days 2) I am progressing on the clickable prototype and am really close to getting it to a stopping point.  I will likely need some more time this week to complete that task, assuming I can shake this illness. 3) I can attend Wednesday zoom tech meeting until 930s 
model answer: ('Wednesday zoom tech meeting',)

Inference:
Question: Who directed Spectre?
Context: Spectre (2015) is the twenty-fourth James Bond film produced by Eon Productions. It features Daniel Craig in his fourth performance as James Bond, and Christoph Waltz as Ernst Stavro Blofeld, with the film marking the character's re-introduction into the series. It was directed by Sam Mendes as his second James Bond film following Skyfall, and was written by John Logan, Neal Purvis, Robert Wade and Jez Butterworth. It is distributed by Metro-Goldwyn-Mayer and Colu

In [3]:
!pip install -q llama-index

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.4.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.8 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 w

In [2]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

ModuleNotFoundError: No module named 'llama_index'

In [ ]:
documents = SimpleDirectoryReader("s3://abb8-ml-model/dailydownload").load_data()

In [4]:
import pandas as pd
df = pd.read_json('s3://abb8-ml-model/dailydownload/mailContent.json')
print(df)

/opt/conda/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


   thread_id        mail_subject  \
0       1001        utility_bill   
1       1002  utility_bill march   

                                           mail_body  
0  Hi Rob, We hope that you’re enjoying your elec...  
1  Hi Rob, We hope that you’re enjoying your elec...  


In [5]:
import pandas as pd
df = pd.read_csv('s3://abb8-ml-model/dailydownload/mailcontents.txt')
print(df)

                                               Hi Rob  \
0       At  Pepco we take pride in making sure tha...   
1       I did want to quickly mention that we have...   
2                                          No worries   
3       If you have any questions don’t hesitate t...   
4   PARENT-TEACHER CONFERENCE REMINDER -springfiel...   
5          Date: ___11/25/2023_______________________   
6   To the parent(s) of __Isabella________________...   
7   Just a reminder that we have a parent-teacher ...   
8   ________Robert  Connor Family______________on ...   
9   in room number __225__________. Our conference...   
10  Please contact the school oce at (____240___)...   
11                   make this scheduled appointment.   
12  I am looking forward to meeting with you and d...   
13                               Your child’s teacher   
14                                      Hello Natasha   
15  Just a friendly reminder about your upcoming a...   
16                             